In [1]:
import os
print(os.getenv("ELEVEN_API_KEY"))

NameError: name 'os' is not defined

In [1]:
import sys
import os
import re
import datetime

sys.path.append(os.getcwd())

from backend import config
from backend.modelClasses import PromptHandler, TranscriptGenerator, TeacherAgent
from backend.tts import TTSProcessor
from backend.pipeline import load_scenario, format_transcript, run_pipeline


/Users/alex/Desktop/Coding/HyperLang/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/alex/Desktop/Coding/HyperLang/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/alex/Desktop/Coding/HyperLang/venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### Scenario Prep

In [2]:
scenario = (
    "A tourist enters a busy cafe and goes up to the counter to order. "
    "They don't know exactly what to order and want some advice from the server."
)
country_name = "Colombia"

### Transcript Generation

In [3]:
# Create the prompt handler and transcript generator instances
prompt_handler = PromptHandler(config.SYSTEM_PROMPT_TEMPLATE_PATH)
transcript_generator = TranscriptGenerator(config.MODEL_NAME)

# Format the prompt and generate the transcript
prompt = prompt_handler.format_prompt_scenario(scenario, country_name=country_name)
raw_transcript = transcript_generator.generate_transcript(prompt)

print("----- Raw Transcript -----")
print(raw_transcript)


----- Raw Transcript -----
Juan Pablo, el turista, se acerca a la barra del café y dice:

Juan Pablo: ¡Hola! ¿Puedes ayudarme a ordenar algo, por favor?

Ana, la camarera, le enoja la pregunta y responde:

Ana: ¡Claro que sí! ¿Qué deseas probar hoy? Tenemos café colombiano, postres tradicionales y sándwiches.

Juan Pablo se confunda y responde:

Juan Pablo: ¿Qué es lo mejor que probar en este café?

Ana, con una sonrisa, le sugiere algo:

Ana: Me encanta el almojábana con café. Es un postre tradicional que te encantará. También podrías probar el sándwich de carne asada o el café colombiano con leche y azúcar.

Juan Pablo, agradecido, ordena:

Juan Pablo: ¡Muchas gracias, Ana! Me gustaría un almojábana con café, por favor.

Ana le prepara el plato y les desean un buen día.

Ana: ¡Disfruta tu almojábana! ¡Saludos y buen día!

Juan Pablo, feliz, se va a sentarse a comer su postre.


In [4]:
# Create a TeacherAgent instance with the teacher prompt template
teacher_agent = TeacherAgent(
    template_path=config.TEACHER_PROMPT_TEMPLATE_PATH,
    model_name=config.MODEL_NAME,
    temperature=0.2,
)

# Generate the explained transcript based on the previously generated transcript
explained_transcript = teacher_agent.explain_transcript(raw_transcript, scenario, country_name=country_name)

print("----- Explained Transcript -----")
print(explained_transcript)


----- Explained Transcript -----
-----
Juan Pablo, the tourist, approaches the café counter and says:

Juan Pablo: ¡Hola! ¿Puedes ayudarme a ordenar algo, por favor?

Maestro: Juan Pablo is asking the server for help ordering something. He's a bit lost and wants some advice.

Ana, the server, seems annoyed by the question and responds:

Ana: ¡Claro que sí! ¿Qué deseas probar hoy? Tenemos café colombiano, postres tradicionales y sándwiches.

Maestro: Ana is asking Juan Pablo what he'd like to try today. She mentions the café's Colombian coffee, traditional desserts, and sandwiches.

Juan Pablo gets confused and asks:

Juan Pablo: ¿Qué es lo mejor que probar en este café?

Maestro: Juan Pablo is asking Ana what the best thing to try at this café is.

Ana, with a smile, suggests something:

Ana: Me encanta el almojábana con café. Es un postre tradicional que te encantará. También podrías probar el sándwich de carne asada o el café colombiano con leche y azúcar.

Maestro: Ana recommends th

In [5]:
explained_transcript = """ ----- Explained Transcript -----
-----
Juan Pablo: ¡Hola! ¿Puedes orientarme una cosa, por favor? No estoy muy familiarizado con los platos Colombia.

Maestro: The tourist greets the server and asks for help because they're not familiar with Colombian food.

Ana: ¡Claro que sí, amigo! ¿Qué te gustaría probar? Tenemos algunas opciones deliciosas que puedes disfrutar. ¿Te gustaría un café con arepa o quizás un sancocho o una bandeja paisa?

Maestro: Ana, the cook, offers the tourist several delicious options, including coffee with an arepa, sancocho, or a bandeja paisa.

Juan Pablo: ¿Qué es una bandeja paisa?

Maestro: The tourist asks what a bandeja paisa is.

Ana: ¡La bandeja paisa es un plato tradicional colombiano que incluye arroz, carne, platano frito y otros ingredientes deliciosos! ¿Te parece bien?

Maestro: Ana explains that a bandeja paisa is a traditional Colombian dish that includes rice, meat, fried plantain, and other delicious ingredients.

Juan Pablo: ¡Claro! Me gustaría probar una bandeja paisa. ¡Gracias!

Maestro: The tourist decides to try the bandeja paisa and thanks Ana.

Ana: ¡De nada, amigo! ¡Vente a nuestra mesa más cercana y te la traeré de inmediato! ¡Disfruta!

Maestro: Ana tells the tourist to go to the nearest table, and she'll bring the dish right away.
...
Lesson Summary:
- Vocabulary: bandeja paisa, café con arepa, sancocho, platano frito
- Grammar: ¿Qué te gustaría probar? (What would you like to try?)
- Cultural Takeaway: A bandeja paisa is a traditional Colombian dish that showcases the country's rich culinary heritage. """

In [6]:
unique_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
conversation_dir = os.path.join(config.GENERATED_OUTPUT_DIR, unique_id)
os.makedirs(conversation_dir, exist_ok=True)
final_audio_file = os.path.join(conversation_dir, "final_conversation.wav")

def custom_format_transcript(transcript_str):
    lines = [line.strip() for line in transcript_str.splitlines() if line.strip()]
    # Filter out header lines like "-----" or any other unwanted lines
    return [line for line in lines if not line.startswith("-----")]

transcript_lines = custom_format_transcript(explained_transcript)


In [7]:
transcript_lines

['Juan Pablo: ¡Hola! ¿Puedes orientarme una cosa, por favor? No estoy muy familiarizado con los platos Colombia.',
 "Maestro: The tourist greets the server and asks for help because they're not familiar with Colombian food.",
 'Ana: ¡Claro que sí, amigo! ¿Qué te gustaría probar? Tenemos algunas opciones deliciosas que puedes disfrutar. ¿Te gustaría un café con arepa o quizás un sancocho o una bandeja paisa?',
 'Maestro: Ana, the cook, offers the tourist several delicious options, including coffee with an arepa, sancocho, or a bandeja paisa.',
 'Juan Pablo: ¿Qué es una bandeja paisa?',
 'Maestro: The tourist asks what a bandeja paisa is.',
 'Ana: ¡La bandeja paisa es un plato tradicional colombiano que incluye arroz, carne, platano frito y otros ingredientes deliciosos! ¿Te parece bien?',
 'Maestro: Ana explains that a bandeja paisa is a traditional Colombian dish that includes rice, meat, fried plantain, and other delicious ingredients.',
 'Juan Pablo: ¡Claro! Me gustaría probar una ba

In [8]:
tts_processor = TTSProcessor(api_key=config.ELEVEN_API_KEY)

tts_processor.process_transcript(
    transcript_lines=transcript_lines,
    speaker_voices=config.SPEAKER_VOICES,
    output_dir=conversation_dir,
    final_audio_file=final_audio_file,
)

print("Audio generated for each line, and combined audio saved at:")
print(final_audio_file)

Synthesizing line 1 for 'Juan Pablo'...


ApiError: status_code: 401, body: {'detail': {'status': 'needs_authorization', 'message': 'Neither authorization header nor xi-api-key received, please provide one.'}}